# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [ ]:
!nvidia-smi

Wed Feb 23 14:42:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the links are dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace, or you can use [the Kaggle API](https://www.kaggle.com/general/74235) to directly download the data into colab.**


### Download train/test metadata

In [3]:
# Main link
!wget -O libriphone.zip "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip"

# Backup Link 0
# !pip install --upgrade gdown
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !pip install --upgrade gdown
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Backup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# Backup link 3
# !wget -O libriphone.zip "https://www.dropbox.com/s/p2ljbtb2bam13in/libriphone.zip?dl=1"

!unzip -q libriphone.zip
!ls libriphone

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'unzip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'ls' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [7]:
!kaggle competitions download -c ml2022spring-hw2


  0%|          | 0.00/458M [00:00<?, ?B/s]
  1%|          | 4.00M/458M [00:00<00:11, 39.9MB/s]
  2%|▏         | 8.00M/458M [00:00<00:12, 39.1MB/s]
  3%|▎         | 12.0M/458M [00:00<00:20, 22.9MB/s]
  3%|▎         | 15.0M/458M [00:00<00:19, 24.1MB/s]
  4%|▍         | 19.0M/458M [00:00<00:16, 28.2MB/s]
  5%|▌         | 23.0M/458M [00:00<00:14, 30.8MB/s]
  6%|▌         | 27.0M/458M [00:00<00:13, 33.0MB/s]
  7%|▋         | 31.0M/458M [00:01<00:13, 32.8MB/s]
  8%|▊         | 35.0M/458M [00:01<00:12, 34.5MB/s]
  9%|▊         | 39.0M/458M [00:01<00:12, 34.5MB/s]
  9%|▉         | 43.0M/458M [00:01<00:12, 33.9MB/s]
 10%|█         | 47.0M/458M [00:01<00:12, 34.0MB/s]
 11%|█         | 51.0M/458M [00:01<00:12, 35.3MB/s]
 12%|█▏        | 55.0M/458M [00:01<00:11, 36.1MB/s]
 13%|█▎        | 59.0M/458M [00:01<00:11, 36.8MB/s]
 14%|█▍        | 63.0M/458M [00:01<00:11, 37.3MB/s]
 15%|█▍        | 67.0M/458M [00:02<00:11, 36.6MB/s]
 16%|█▌        | 71.0M/458M [00:02<00:10, 37.2MB/s]
 16%|█▋        | 75.

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [1]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm
import numpy as np

In [2]:
# Import Utils via util.py
# import util
# import importlib
# importlib.reload(util)
from util import *

## Define Dataset

In [3]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model
Via model.py

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# import importlib
# import model
# importlib.reload(model)
from model import *

## Hyper-parameters

In [6]:
# data prarameters
concat_nframes = 27              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 94873                        # random seed
batch_size = 1024                # batch size
num_epoch = 100                # the number of training epoch
learning_rate = 0.001          # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 7               # the number of hidden layers
hidden_dim = 512                # the hidden dim
dropout_rate = 0.25              # define dropout rate

## Prepare dataset and model

In [7]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


3428it [00:57, 59.23it/s]


[INFO] train set
torch.Size([2116368, 1053])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:16, 52.63it/s]

[INFO] val set
torch.Size([527790, 1053])
torch.Size([527790])


In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# fix random seed
same_seeds(seed)

DEVICE: cuda:0


In [9]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim, dropout_rate=dropout_rate).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## Training

In [10]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)
                
                loss = criterion(outputs, labels) 
                
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


100%|██████████| 516/516 [00:05<00:00, 88.54it/s]


[001/100] Train Acc: 0.598848 Loss: 1.333314 | Val Acc: 0.669876 loss: 1.062176
saving model with acc 0.670


100%|██████████| 516/516 [00:05<00:00, 95.43it/s] 


[002/100] Train Acc: 0.662474 Loss: 1.098703 | Val Acc: 0.696777 loss: 0.967313
saving model with acc 0.697


100%|██████████| 516/516 [00:05<00:00, 91.57it/s]


...

[098/100] Train Acc: 0.804271 Loss: 0.608290 | Val Acc: 0.760376 loss: 0.793652


100%|██████████| 516/516 [00:05<00:00, 91.49it/s] 


[099/100] Train Acc: 0.804204 Loss: 0.606957 | Val Acc: 0.760363 loss: 0.791765


100%|██████████| 516/516 [00:05<00:00, 89.72it/s] 

[100/100] Train Acc: 0.804403 Loss: 0.607367 | Val Acc: 0.760196 loss: 0.790206


In [11]:
del train_loader, val_loader
gc.collect()

22

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [12]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:17, 61.99it/s]

[INFO] test set
torch.Size([646268, 1053])


In [13]:
# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [14]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 632/632 [00:06<00:00, 101.14it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [15]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))